In [ ]:
import os
import json
import tempfile
import re
import google.generativeai as genai
import gspread
import mimetypes
from google.oauth2.service_account import Credentials
from openpyxl.utils import get_column_letter
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file",
]

CREDS_FILE = os.getenv("CREDS_JSON_ENV")
DEFAULT_SHEET_ID = "17tMHStXQYXaIQHQIA4jdUyHaYt_tuoNCEEuJCstWEuw"

COMPANY_NAME_ROW = 1
CONTACT_INFO_ROW = 2
HEADER_ROW = 3
ITEM_MASTER_LIST_COL = 2
COLUMNS_PER_SUPPLIER = 4

prompt = """# System Message for Product List Extraction (PDF/Text Table Processing)
## CRITICAL: ANTI-HALLUCINATION WARNING
You MUST ONLY extract information that is EXPLICITLY visible in the document. 
- DO NOT add data from anywhere other than the one in the uploaded document. Adding data that is not from the document is a serious mistake.
- DO NOT create or invent any products, prices, or specifications
- DO NOT add data from the attached Example in the prompt.
- DO NOT add products that are not clearly listed as distinct line items
- DO NOT attempt to break down a single product into multiple products
- DO NOT interpret descriptive text as separate products
- If uncertain about any information, LEAVE IT OUT rather than guessing

## CRITICAL: CONTACT INFORMATION EXTRACTION
Pay special attention to contact information in the document header or footer:
- Extract any email addresses (example@domain.com)
- Extract any phone numbers (formats like 02-3384825, 081-1234567, 095-525-2623)
- Put email and phone details in the "contact" field
- Format: "Email: email@example.com, Phone: 081-234-5678"
- Use only the phone number and email address on the letterhead. Do not add "บริษัท ลูก้า แอสเซท จำกัด, 081-781-7283" contact information

## CRITICAL: COMPLETE EXTRACTION REQUIREMENT
You MUST extract ALL products visible in the document:
- Extract EVERY product line item visible in the document
- Preserve hierarchical structure (groups/categories) of products if present
- Ensure NO products are missed or skipped
- Each row with a distinct price is one product

## Input Format
Provide PDF files (or images/tables with text extraction) containing product information (receipts, invoices, product lists, etc.).

## Task
Extract ALL product information EXPLICITLY visible in the document:
- Extract products with quantities, units, and prices
- Preserve parent-child relationships (main categories and sub-items)
- Maintain hierarchical product groupings (numbered sections, categories)
- Also extract additional quotation details like price validity, delivery time, payment terms, etc.

## Hierarchical Structure Handling
Many quotations organize products hierarchically. When you see this:
- Include category names in product descriptions (e.g., "งานบันไดกระจก งานพื้นตก - กระจกเทมเปอร์ใส หนา 10 มม. ขนาด 4.672×0.97 ม.")
- If product descriptions begin with numbers (1, 2, 3...), REMOVE those numbers
- Include all parent category information in each product's name without the leading numbers

## Output Format (JSON only)
You must return ONLY this JSON structure:
{
  "company": "company name or first name + last name (NEVER null)",
  "vat": true,
  "name": "customer name or null",
  "contact": "phone number or email or null",
  "priceGuaranteeDay": 30
  "deliveryTime": "",
  "paymentTerms": "",
  "otherNotes": "",
  "products": [
    {
      "name": "full product description including ALL parent category info, specifications AND dimensions WITHOUT leading numbers",
      "quantity": 1,
      "unit": "match the unit shown in the document (e.g., แผ่น, ตร.ม., ชิ้น, ตัว, เมตร, ชุด)",
      "pricePerUnit": 0,
      "totalPrice": 0
    }
  ],
  "totalPrice": 0,
  "totalVat": 0,
  "totalPriceIncludeVat": 0
}

## Example 1 (Format with Item/ART.No./Description/Qty/Unit/Price columns):
| Item | ART.No. | Description | Qty | Unit | Standard Price | Discount Price | Amount |
|------|---------|-------------|-----|------|----------------|---------------|--------|
| 1    | CPW-xxxx| SPC ลายไม้ 4.5 มิล (ก้างปลา) | 1.00 | ตร.ม. |  | 520.00 | 520.000 |
| 2    |         | ค่าแรงติดตั้ง | 1.00 | ตร.ม. |  | 150.00 | 150.000 |

## Example 2 (Format with ลำดับ/รหัสสินค้า/รายละเอียดสินค้า columns):
| ลำดับ | รหัสสินค้า | รายละเอียดสินค้า | หน่วย | จำนวน | ราคา/หน่วย(บาท) | จำนวนเงิน(บาท) |
|------|---------|----------------|------|------|--------------|------------|
| 1    |         | พื้นไม้ไวนิลลายไม้ปลา 4.5 มม. LKT 4.5 mm x 0.3 mm สีฟ้าเซอร์คูลี (1 กล่อง บรรจุ 18 แผ่น หรือ 1.3 ตร.ม) | ตร.ม. | 1.30 | 680.00 | 884.00 |

## Field Extraction Guidelines

### name (Product Description)
* CRITICAL: Include ALL hierarchical information in each product name:
  - Category names/headings (e.g., "งานบันไดกระจก งานพื้นตก")
  - Sub-category information (e.g., "เหล็กตัวซีชุบสังกะสี")
  - Glass type, thickness (e.g., "กระจกเทมเปอร์ใส หนา 10 มม.")
  - Exact dimensions (e.g., "ขนาด 4.672×0.97 ม.")
* REMOVE any leading numbers (1., 2., 3.) from the product descriptions
* Format hierarchical products as: "[Category Name] - [Material] - [Type] - [Dimensions]"
* Include: ALL distinguishing characteristics that make each product unique
* Example: "งานบันไดกระจก งานพื้นตก - เหล็กตัวซีชุบสังกะสี ไม่รวมปูน - กระจกเทมเปอร์ใส หนา 10 มม. ขนาด 4.672×0.97 ม."

### unit and quantity (DIRECT EXTRACTION RULE)
* Extract unit and quantity DIRECTLY from each line item as shown
* Use the exact unit shown in the document (ชุด, แผ่น, ตร.ม., ชิ้น, ตัว, เมตร, etc.)
* Extract the exact quantity shown for each product (never assume or calculate)
* NEVER create quantities or units that aren't explicitly shown in the document
* Pay special attention to decimal quantities - extract the full decimal precision

### pricePerUnit and totalPrice
* Extract ONLY prices clearly visible in the document
* Use numeric values only (no currency symbols)
* Extract cleanly from pricing fields as shown in each line item
* NEVER calculate or estimate prices that aren't explicitly shown
* NEVER combine different products' prices
* Pay special attention to decimal prices - extract the EXACT decimal values shown

### Additional Quotation Details
* Extract these additional fields if present:
  - "กำหนดยืนราคา (วัน)", "กำหนดยืนราคา", "การยืนราคา" - Price validity period in days (priceGuaranteeDay)
  - "ระยะเวลาส่งมอบสินค้าหลังจากได้รับ PO" - Delivery time after PO (deliveryTime)
  - "การชำระเงิน" - Payment terms (paymentTerms)
  - "อื่น ๆ" - Other notes (otherNotes)
* Extract as text exactly as written, preserving numbers and Thai language

### CRITICAL: Pricing summaries and summary values
* Extract the exact values for these three summary items:
  - "รวมเป็นเงิน" - the initial subtotal (totalPrice)
  - "ภาษีมูลค่าเพิ่ม 7%" - the VAT amount (totalVat)
  - "ยอดรวมทั้งสิ้น" - the final total (totalPriceIncludeVat)
* Alternative labels to match:
  - For totalPrice: "รวม", "รวมเป็นเงิน", "ราคารวม", "Total", "TOTAL AMOUNT", "รวมราคา"
  - For totalVat: "ภาษีมูลค่าเพิ่ม 7%", "VAT 7%"
  - For totalPriceIncludeVat: "ยอดรวมทั้งสิ้น", "รวมทั้งหมด", "รวมเงินทั้งสิน", "ราคารวมสุทธิ", "รวมราคางานทั้งหมดตามสัญญา"
* Extract the exact values as shown (remove commas, currency symbols)
* CRITICAL: Preserve full decimal precision in all monetary values

## FINAL VERIFICATION
Review the extracted products one last time and verify:
1. Count the number of products you've extracted
2. Verify this matches EXACTLY with the number of product rows visible in the document
3. Check that ALL products have proper hierarchical information included WITHOUT leading numbers
4. Ensure NO products are missing - every line item with a price must be extracted
5. Confirm all dimensions and specifications are preserved correctly
6. Verify all decimal values (quantities and prices) maintain their full precision
"""

image_prompt = """# System Message for Product List Extraction from Images
## CRITICAL: ANTI-HALLUCINATION WARNING
You MUST ONLY extract information that is EXPLICITLY visible in the image. 
- DO NOT create or invent any products, prices, or specifications
- DO NOT add products that are not clearly listed as distinct line items
- DO NOT interpret descriptive text as separate products
- If text is unclear or unreadable, mark it as uncertain rather than guessing

## CRITICAL: COMPLETE EXTRACTION REQUIREMENT
You MUST extract ALL products visible in the image:
- Extract EVERY product line item visible in the image
- Preserve hierarchical structure (groups/categories) of products if present
- Ensure NO products are missed or skipped
- Each row with a distinct price is one product

## Input Format
I'm providing an image of a document containing product information.

## Task
Extract ALL product information EXPLICITLY visible in the image:
- Extract products with quantities, units, and prices
- Preserve parent-child relationships (main categories and sub-items)
- Maintain hierarchical product groupings (numbered sections, categories)
- Also extract additional quotation details like price validity, delivery time, payment terms, etc.

## Hierarchical Structure Handling
Many quotations organize products hierarchically. When you see this:
- Include category names in product descriptions (e.g., "งานบันไดกระจก งานพื้นตก - กระจกเทมเปอร์ใส หนา 10 มม. ขนาด 4.672×0.97 ม.")
- If product descriptions begin with numbers (1, 2, 3...), REMOVE those numbers
- Include all parent category information in each product's name without the leading numbers

## Output Format (JSON only)
You must return ONLY this JSON structure:
{
  "company": "company name or first name + last name (NEVER null)",
  "vat": true,
  "contact": "phone number or email or null",
  "priceGuaranteeDay": 30
  "deliveryTime": "",
  "paymentTerms": "",
  "otherNotes": "",
  "products": [
    {
      "name": "full product description including ALL parent category info, specifications AND dimensions WITHOUT leading numbers",
      "quantity": 1,
      "unit": "match the unit shown in the document (e.g., แผ่น, ตร.ม., ชิ้น, ตัว, เมตร, ชุด)",
      "pricePerUnit": 0,
      "totalPrice": 0
    }
  ],
  "totalPrice": 0,
  "totalVat": 0,
  "totalPriceIncludeVat": 0
}


## Example 1 (Format with Item/ART.No./Description/Qty/Unit/Price columns):
| Item | ART.No. | Description | Qty | Unit | Standard Price | Discount Price | Amount |
|------|---------|-------------|-----|------|----------------|---------------|--------|
| 1    | CPW-xxxx| SPC ลายไม้ 4.5 มิล (ก้างปลา) | 1.00 | ตร.ม. |  | 520.00 | 520.000 |
| 2    |         | ค่าแรงติดตั้ง | 1.00 | ตร.ม. |  | 150.00 | 150.000 |

## Example 2 (Format with ลำดับ/รหัสสินค้า/รายละเอียดสินค้า columns):
| ลำดับ | รหัสสินค้า | รายละเอียดสินค้า | หน่วย | จำนวน | ราคา/หน่วย(บาท) | จำนวนเงิน(บาท) |
|------|---------|----------------|------|------|--------------|------------|
| 1    |         | พื้นไม้ไวนิลลายไม้ปลา 4.5 มม. LKT 4.5 mm x 0.3 mm สีฟ้าเซอร์คูลี (1 กล่อง บรรจุ 18 แผ่น หรือ 1.3 ตร.ม) | ตร.ม. | 1.30 | 680.00 | 884.00 |


## Field Extraction Guidelines

### name (Product Description)
* CRITICAL: Include ALL hierarchical information in each product name:
  - Category names/headings (e.g., "งานบันไดกระจก งานพื้นตก")
  - Sub-category information (e.g., "เหล็กตัวซีชุบสังกะสี")
  - Glass type, thickness (e.g., "กระจกเทมเปอร์ใส หนา 10 มม.")
  - Exact dimensions (e.g., "ขนาด 4.672×0.97 ม.")
* REMOVE any leading numbers (1., 2., 3.) from the product descriptions
* Format hierarchical products as: "[Category Name] - [Material] - [Type] - [Dimensions]"
* Include: ALL distinguishing characteristics that make each product unique
* Example: "งานบันไดกระจก งานพื้นตก - เหล็กตัวซีชุบสังกะสี ไม่รวมปูน - กระจกเทมเปอร์ใส หนา 10 มม. ขนาด 4.672×0.97 ม."

### unit and quantity (DIRECT EXTRACTION RULE)
* Extract unit and quantity DIRECTLY from each line item as shown
* Use the exact unit shown in the document (ชุด, แผ่น, ตร.ม., ชิ้น, ตัว, เมตร, จำนวนต่อชุด etc.)
* Extract the exact quantity shown for each product (never assume or calculate)
* NEVER create quantities or units that aren't explicitly shown in the document
* Pay special attention to decimal quantities - extract the full decimal precision

### pricePerUnit and totalPrice
* Extract ONLY prices clearly visible in the document
* Use numeric values only (no currency symbols)
* Extract cleanly from pricing fields as shown in each line item
* NEVER calculate or estimate prices that aren't explicitly shown
* NEVER combine different products' prices
* Pay special attention to decimal prices - extract the EXACT decimal values shown

### Additional Quotation Details
* Extract these additional fields if present:
  - "กำหนดยืนราคา (วัน)", "กำหนดยืนราคา", "การยืนราคา" - Price validity period in days (priceGuaranteeDay)
  - "ระยะเวลาส่งมอบสินค้าหลังจากได้รับ PO" - Delivery time after PO (deliveryTime)
  - "การชำระเงิน" - Payment terms (paymentTerms)
  - "อื่น ๆ" - Other notes (otherNotes)
* Extract as text exactly as written, preserving numbers and Thai language

### CRITICAL: Pricing summaries and summary values
* Extract the exact values for these three summary items:
  - "รวมเป็นเงิน" - the initial subtotal (totalPrice)
  - "ภาษีมูลค่าเพิ่ม 7%" - the VAT amount (totalVat)
  - "ยอดรวมทั้งสิ้น" - the final total (totalPriceIncludeVat)
* Alternative labels to match:
  - For totalPrice: "รวม", "รวมเป็นเงิน", "ราคารวม", "Total", "TOTAL AMOUNT", "รวมราคา"
  - For totalVat: "ภาษีมูลค่าเพิ่ม 7%", "VAT 7%"
  - For totalPriceIncludeVat: "ยอดรวมทั้งสิ้น", "รวมทั้งหมด", "รวมเงินทั้งสิน", "ราคารวมสุทธิ", "รวมราคางานทั้งหมดตามสัญญา"
* Extract the exact values as shown (remove commas, currency symbols)
* CRITICAL: Preserve full decimal precision in all monetary values

## FINAL VERIFICATION
Review the extracted products one last time and verify:
1. Count the number of products you've extracted
2. Verify this matches EXACTLY with the number of product rows visible in the image
3. Check that ALL products have proper hierarchical information included WITHOUT leading numbers
4. Ensure NO products are missing - every line item with a price must be extracted
5. Confirm all dimensions and specifications are preserved correctly
6. Verify all decimal values (quantities and prices) maintain their full precision
"""

validation_prompt = """
You are a data validation expert specializing in Thai construction quotations.
I've extracted product data from a document, but there may be missing products or hierarchical relationships.

## CRITICAL: COMPLETE DATA CHECK
Your primary task is to ensure ALL products are correctly extracted with their hierarchical structure:
1. Check that all products visible in the document have been extracted
2. Ensure parent-child relationships and category groupings are preserved
3. Verify that all products have complete descriptions including their category name
4. Make sure no products are missing dimensions or specifications

## CRITICAL: PRESERVE PRODUCT HIERARCHY
Thai construction quotations often organize products hierarchically by categories:
- Category names with descriptive details
- Materials and specifications
- Dimensions

Each product must include its complete hierarchy:
"[Category Name] - [Material] - [Type] - [Dimensions]"

Examples (DO NOT add data from the attached Example in the prompt): 
- "งานบันไดกระจก งานพื้นตก - เหล็กตัวซีชุบสังกะสี ไม่รวมปูน - กระจกเทมเปอร์ใส หนา 10 มม. ขนาด 4.672×0.97 ม."
- "งานพื้นตก (ชั้นลอย) - เหล็กตัวซีชุบสังกะสี ไม่รวมปูน - เทมเปอร์ใส หนา 10 มม. ขนาด 3.565×0.97 ม."

## CRITICAL: DECIMAL NUMBER ACCURACY
Pay special attention to:
1. Quantities with decimals (extract full precision)
2. Dimensions with decimals (preserve exact measurements)
3. Prices with decimals (maintain exact values)

## CRITICAL: CLEAN PRODUCT DESCRIPTIONS
1. REMOVE any leading numbers (1., 2., 3., etc.) from product descriptions
2. Ensure NO product descriptions begin with numbering
3. Maintain all other hierarchical information and details

Review the data carefully and FIX these issues:
1. ADD any missing products that should be extracted from the source document
2. FIX product names to include complete hierarchical information WITHOUT leading numbers
3. ENSURE all dimensions and specifications are preserved with full decimal precision
4. VERIFY every product has the correct quantity, unit, price and total with full decimal precision

Original extraction:
{extracted_json}

Return ONLY a valid JSON object with no explanations.
"""

matching_prompt = """
You are a product matching expert specializing in construction materials in Thailand.

## TASK
Match products from a target list (new quotation) to a reference list (master product list) based on their attributes, materials, dimensions, and specifications.

## CRITICAL MATCHING RULES
1. Focus on the meaning and specifications, not just text similarity
2. Consider materials, dimensions, thickness, and product type as key matching factors
3. Each reference product can only be matched ONCE (never match the same reference item to multiple target items)
4. If a product cannot be matched with high confidence (>70%), leave it unmatched

## UNIT CONVERSION AWARENESS
Pay special attention to dimensions and units:
- Convert between mm, cm, and m when comparing dimensions (1m = 100cm = 1000mm)
- Match items with similar dimensions even if units differ (e.g., "4672x970 mm" and "4.672x0.97 m" are the same)
- Consider products like "ราวกันตกฝังปูน" with similar specifications as potential matches even if dimensions vary slightly

## INPUT
- Target Products: New products from a quotation that need to be matched
- Reference Products: Existing master list of products to match against

## MATCHING CRITERIA (in priority order)
1. Material type match (e.g., glass with glass, steel with steel)
2. Dimensions match (within 5% tolerance, after unit conversion)
3. Thickness match (within 5% tolerance, after unit conversion)
4. Product type/category match

## Material Type Examples
- Glass: กระจก, glass, tempered, เทมเปอร์
- Steel: เหล็ก, steel, galvanized, ชุบสังกะสี
- Aluminum: อลูมิเนียม, aluminum, aluminium
- Wood: ไม้, wood, timber, plywood

## OUTPUT FORMAT
Return a JSON object with these properties:
{{
  "matchedItems": [
    {{
      "name": "reference product name",
      "quantity": target quantity,
      "unit": target unit,
      "pricePerUnit": target price per unit,
      "totalPrice": target total price
    }}
  ],
  "uniqueItems": [
    {{
      "name": "target product name",
      "quantity": target quantity,
      "unit": target unit,
      "pricePerUnit": target price per unit,
      "totalPrice": target total price
    }}
  ]
}}

Where:
- matchedItems: Array of products that found a match in the reference list
  - Use the reference product name, but the target's quantity, unit, and prices
- uniqueItems: Array of target products that couldn't be matched plus any unused reference products

## Target Products:
{target_products}

## Reference Products:
{reference_products}
"""


def extract_sheet_id_from_url(url):
    if not url:
        return None
    if "/" not in url and " " not in url and len(url) > 20:
        return url
    m = re.search(r"spreadsheets/d/([a-zA-Z0-9-_]+)", url)
    return m.group(1) if m else None


def extract_json_from_text(text):
    start = text.find("{")
    end = text.rfind("}") + 1
    if start >= 0 and end > start:
        json_str = text[start:end]
        cleaned_json = re.sub(r",\s*}", "}", json_str)
        cleaned_json = re.sub(r",\s*]", "]", cleaned_json)
        try:
            return json.loads(cleaned_json)
        except json.JSONDecodeError:
            return None
    return None


def extract_contact_info(text):
    if not text:
        return ""
    phone_pattern = r"(?<!\w)((0\d{1,2}[-\s]?\d{3}[-\s]?\d{3,4})|(0\d{2}[-\s]?\d{7})|(0\d{2}[-\s]?\d{3}[-\s]?\d{4}))(?!\w)"
    email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
    phone_matches = re.findall(phone_pattern, text)
    email_matches = re.findall(email_pattern, text)
    phone_numbers = [m[0] for m in phone_matches] if phone_matches else []
    contact_parts = []
    if email_matches:
        contact_parts.append(f"Email: {', '.join(email_matches)}")
    if phone_numbers:
        formatted_phones = []
        for phone in phone_numbers:
            clean_phone = re.sub(r"\s", "", phone)
            if len(clean_phone) >= 9:
                formatted_phones.append(clean_phone)
        contact_parts.append(f"Phone: {', '.join(formatted_phones)}")
    return ", ".join(contact_parts)


def clean_product_name(name):
    if not name:
        return "Unknown Product"
    return re.sub(r"^\d+\.\s*", "", name.strip())


def validate_json_data(json_data):
    if not json_data:
        return {
            "company": "Unknown Company",
            "contact": "",
            "products": [],
            "totalPrice": 0,
            "totalVat": 0,
            "totalPriceIncludeVat": 0,
            "priceGuaranteeDay": 0,
            "deliveryTime": "",
            "paymentTerms": "",
            "otherNotes": "",
        }
    if not json_data.get("company"):
        json_data["company"] = "Unknown Company"
    if "contact" in json_data:
        if isinstance(json_data["contact"], dict):
            contact_parts = []
            if "email" in json_data["contact"]:
                contact_parts.append(f"Email: {json_data['contact']['email']}")
            if "phone" in json_data["contact"]:
                contact_parts.append(f"Phone: {json_data['contact']['phone']}")
            json_data["contact"] = ", ".join(contact_parts)
        else:
            json_data["contact"] = extract_contact_info(str(json_data["contact"]))
    else:
        json_data["contact"] = ""
    if not json_data.get("products"):
        json_data["products"] = []
    for product in json_data.get("products", []):
        if product.get("name"):
            product["name"] = clean_product_name(product["name"])
        else:
            product["name"] = "Unknown Product"
        if not product.get("quantity"):
            product["quantity"] = 1
        else:
            product["quantity"] = (
                float(str(product["quantity"]).replace(",", ""))
                if str(product["quantity"])
                .replace(",", "")
                .replace(".", "", 1)
                .isdigit()
                else 1
            )
        if not product.get("unit"):
            product["unit"] = "ชิ้น"
        if not product.get("pricePerUnit"):
            product["pricePerUnit"] = 0
        else:
            product["pricePerUnit"] = (
                float(str(product["pricePerUnit"]).replace(",", ""))
                if str(product["pricePerUnit"])
                .replace(",", "")
                .replace(".", "", 1)
                .isdigit()
                else 0
            )
        if not product.get("totalPrice"):
            product["totalPrice"] = round(
                product["quantity"] * product["pricePerUnit"], 2
            )
        else:
            product["totalPrice"] = (
                float(str(product["totalPrice"]).replace(",", ""))
                if str(product["totalPrice"])
                .replace(",", "")
                .replace(".", "", 1)
                .isdigit()
                else round(product["quantity"] * product["pricePerUnit"], 2)
            )
    if not json_data.get("totalPrice"):
        json_data["totalPrice"] = sum(
            p.get("totalPrice", 0) for p in json_data.get("products", [])
        )
    else:
        json_data["totalPrice"] = (
            float(str(json_data["totalPrice"]).replace(",", ""))
            if str(json_data["totalPrice"])
            .replace(",", "")
            .replace(".", "", 1)
            .isdigit()
            else sum(p.get("totalPrice", 0) for p in json_data.get("products", []))
        )
    if not json_data.get("totalVat"):
        json_data["totalVat"] = round(json_data["totalPrice"] * 0.07, 2)
    else:
        json_data["totalVat"] = (
            float(str(json_data["totalVat"]).replace(",", ""))
            if str(json_data["totalVat"]).replace(",", "").replace(".", "", 1).isdigit()
            else round(json_data["totalPrice"] * 0.07, 2)
        )
    if not json_data.get("totalPriceIncludeVat"):
        json_data["totalPriceIncludeVat"] = round(
            json_data["totalPrice"] + json_data["totalVat"], 2
        )
    else:
        json_data["totalPriceIncludeVat"] = (
            float(str(json_data["totalPriceIncludeVat"]).replace(",", ""))
            if str(json_data["totalPriceIncludeVat"])
            .replace(",", "")
            .replace(".", "", 1)
            .isdigit()
            else round(json_data["totalPrice"] + json_data["totalVat"], 2)
        )
    if "priceGuaranteeDay" not in json_data:
        json_data["priceGuaranteeDay"] = 0
    if "deliveryTime" not in json_data:
        json_data["deliveryTime"] = ""
    if "paymentTerms" not in json_data:
        json_data["paymentTerms"] = ""
    if "otherNotes" not in json_data:
        json_data["otherNotes"] = ""
    return json_data


def enhance_with_gemini(json_data):
    if not json_data:
        return None
    validation_formatted = validation_prompt.format(
        extracted_json=json.dumps(json_data, ensure_ascii=False)
    )
    model = genai.GenerativeModel(model_name="gemini-2.5-flash")
    response = model.generate_content(validation_formatted)
    enhanced_text = response.text.strip()
    if "```" in enhanced_text:
        blocks = re.findall(r"```(?:json)?(.*?)```", enhanced_text, re.DOTALL)
        if blocks:
            enhanced_text = blocks[0].strip()
    enhanced_data = json.loads(enhanced_text) if enhanced_text else json_data
    if not isinstance(enhanced_data, dict):
        enhanced_data = extract_json_from_text(enhanced_text)
        return enhanced_data if enhanced_data else json_data
    return enhanced_data


def match_products_with_gemini(target_products, reference_products):
    if not target_products or not reference_products:
        return {"matchedItems": [], "uniqueItems": target_products or []}
    match_prompt_formatted = matching_prompt.format(
        target_products=json.dumps(target_products, ensure_ascii=False),
        reference_products=json.dumps(reference_products, ensure_ascii=False),
    )
    model = genai.GenerativeModel(
        model_name="gemini-2.5-pro",
        generation_config={"temperature": 0.1, "top_p": 0.95},
    )
    response = model.generate_content(match_prompt_formatted)
    match_text = response.text.strip()
    if "```" in match_text:
        blocks = re.findall(r"```(?:json)?(.*?)```", match_text, re.DOTALL)
        if blocks:
            match_text = blocks[0].strip()
    match_data = None
    try:
        match_data = json.loads(match_text)
    except json.JSONDecodeError:
        match_data = extract_json_from_text(match_text)
    if not match_data:
        return {"matchedItems": [], "uniqueItems": target_products}
    if "matchedItems" not in match_data:
        match_data["matchedItems"] = []
    if "uniqueItems" not in match_data:
        match_data["uniqueItems"] = target_products
    return match_data


def authenticate_and_open_sheet(sheet_id):
    creds = Credentials.from_service_account_file(CREDS_FILE, scopes=SCOPES)
    client = gspread.authorize(creds)
    return client.open_by_key(sheet_id).get_worksheet(0)


def ensure_first_three_rows_exist(ws):
    p = []
    for i in range(1, 4):
        p.append({"range": f"A{i}:B{i}", "values": [["", ""]]})
    ws.batch_update(p, value_input_option="USER_ENTERED")


def find_next_available_column(ws):
    vals = ws.get_all_values()
    m = ITEM_MASTER_LIST_COL
    for row in vals[:HEADER_ROW]:
        for i, c in enumerate(row):
            if c.strip():
                m = max(m, i + 1)
    return m + 1


def update_google_sheet_for_single_file(ws, data):
    ensure_first_three_rows_exist(ws)
    start_row = HEADER_ROW + 1
    sheet_values = ws.get_all_values()
    existing_products = []
    SUMMARY_LABELS = [
        "รวมเป็นเงิน",
        "ภาษีมูลค่าเพิ่ม 7%",
        "ยอดรวมทั้งสิ้น",
        "กำหนดยืนราคา (วัน)",
        "ระยะเวลาส่งมอบสินค้าหลังจากได้รับ PO",
        "การชำระเงิน",
        "อื่น ๆ",
    ]
    summary_row_map = {}
    first_summary_row = -1

    for row_idx, row in enumerate(sheet_values[HEADER_ROW:], start=start_row):
        if len(row) >= ITEM_MASTER_LIST_COL and row[ITEM_MASTER_LIST_COL - 1].strip():
            cell_value = row[ITEM_MASTER_LIST_COL - 1].strip()
            if cell_value in SUMMARY_LABELS:
                if first_summary_row == -1:
                    first_summary_row = row_idx
                summary_row_map[cell_value] = row_idx
            else:
                product_name = clean_product_name(cell_value)
                existing_products.append({"name": product_name, "row": row_idx})

    existing_suppliers = {}
    for col_idx in range(
        ITEM_MASTER_LIST_COL,
        len(sheet_values[0]) if sheet_values else 0,
        COLUMNS_PER_SUPPLIER,
    ):
        if COMPANY_NAME_ROW - 1 < len(sheet_values) and col_idx < len(
            sheet_values[COMPANY_NAME_ROW - 1]
        ):
            supplier_name = sheet_values[COMPANY_NAME_ROW - 1][col_idx].strip()
            if supplier_name:
                existing_suppliers[supplier_name] = col_idx

    next_avail_col = find_next_available_column(ws)

    products = data.get("products", [])
    if not products:
        return 0

    for product in products:
        if product.get("name"):
            product["name"] = clean_product_name(product["name"])

    company_name = data.get("company", "Unknown Company")
    col_idx = existing_suppliers.get(company_name, next_avail_col)
    if col_idx == next_avail_col:
        next_avail_col += COLUMNS_PER_SUPPLIER
    batch_requests = [
        {
            "range": f"{get_column_letter(col_idx)}{COMPANY_NAME_ROW}",
            "values": [[company_name]],
        },
        {
            "range": f"{get_column_letter(col_idx)}{CONTACT_INFO_ROW}",
            "values": [[f"{data.get('contact','')}".strip()]],
        },
        {
            "range": f"{get_column_letter(col_idx)}{HEADER_ROW}:{get_column_letter(col_idx+COLUMNS_PER_SUPPLIER-1)}{HEADER_ROW}",
            "values": [["ปริมาณ", "หน่วย", "ราคาต่อหน่วย", "รวมเป็นเงิน"]],
        },
    ]

    reference_data = [{"name": item["name"]} for item in existing_products]
    match_results = match_products_with_gemini(products, reference_data)
    matched_items = match_results["matchedItems"]
    unique_items = match_results["uniqueItems"]

    populated_rows = set()
    for item in matched_items:
        item_name = item["name"]
        for existing in existing_products:
            if existing["name"] == item_name and existing["row"] not in populated_rows:
                batch_requests.append(
                    {
                        "range": f"{get_column_letter(col_idx)}{existing['row']}:{get_column_letter(col_idx+COLUMNS_PER_SUPPLIER-1)}{existing['row']}",
                        "values": [
                            [
                                item.get("quantity", 1),
                                item.get("unit", "ชิ้น"),
                                item.get("pricePerUnit", 0),
                                item.get("totalPrice", 0),
                            ]
                        ],
                    }
                )
                populated_rows.add(existing["row"])
                break

    new_products = []
    for item in unique_items:
        if isinstance(item, dict) and "name" in item and "quantity" in item:
            item["name"] = clean_product_name(item["name"])

            if not any(
                existing["name"] == item["name"] for existing in existing_products
            ):
                new_products.append(item)

    insertion_row = (
        first_summary_row
        if first_summary_row > 0
        else (start_row + len(existing_products))
    )

    if new_products:
        new_rows = []
        for _ in range(len(new_products)):
            new_rows.append([""] * ws.col_count)

        if new_rows:
            ws.insert_rows(new_rows, insertion_row)

        row_shift = len(new_products)
        if first_summary_row > 0:
            for label in summary_row_map:
                summary_row_map[label] += row_shift

        for i, product in enumerate(new_products):
            row = insertion_row + i
            batch_requests.append(
                {
                    "range": f"{get_column_letter(ITEM_MASTER_LIST_COL)}{row}",
                    "values": [[product.get("name", "Unknown Product")]],
                }
            )

            batch_requests.append(
                {
                    "range": f"{get_column_letter(col_idx)}{row}:{get_column_letter(col_idx+COLUMNS_PER_SUPPLIER-1)}{row}",
                    "values": [
                        [
                            product.get("quantity", 1),
                            product.get("unit", "ชิ้น"),
                            product.get("pricePerUnit", 0),
                            product.get("totalPrice", 0),
                        ]
                    ],
                }
            )

    price_col = col_idx + COLUMNS_PER_SUPPLIER - 1
    summary_items = [
        ("รวมเป็นเงิน", data.get("totalPrice", 0)),
        ("ภาษีมูลค่าเพิ่ม 7%", data.get("totalVat", 0)),
        ("ยอดรวมทั้งสิ้น", data.get("totalPriceIncludeVat", 0)),
        ("กำหนดยืนราคา (วัน)", data.get("priceGuaranteeDay", "")),
        ("ระยะเวลาส่งมอบสินค้าหลังจากได้รับ PO", data.get("deliveryTime", "")),
        ("การชำระเงิน", data.get("paymentTerms", "")),
        ("อื่น ๆ", data.get("otherNotes", "")),
    ]

    if summary_row_map:

        for label, value in summary_items:
            if label in summary_row_map:
                batch_requests.append(
                    {
                        "range": f"{get_column_letter(price_col)}{summary_row_map[label]}",
                        "values": [[value]],
                    }
                )
    else:
        summary_row = insertion_row + len(new_products) + 2
        for i, (label, value) in enumerate(summary_items):
            row = summary_row + i
            batch_requests.append(
                {
                    "range": f"{get_column_letter(ITEM_MASTER_LIST_COL)}{row}",
                    "values": [[label]],
                }
            )
            batch_requests.append(
                {"range": f"{get_column_letter(price_col)}{row}", "values": [[value]]}
            )

    if batch_requests:
        ws.batch_update(batch_requests, value_input_option="USER_ENTERED")

    return 1


def update_google_sheet_with_multiple_files(ws, all_json_data):
    if len(all_json_data) == 1:
        return update_google_sheet_for_single_file(ws, all_json_data[0])

    ensure_first_three_rows_exist(ws)
    start_row = HEADER_ROW + 1
    sheet_values = ws.get_all_values()
    existing_products = []
    matched_product_rows = set()

    for row_idx, row in enumerate(sheet_values[HEADER_ROW:], start=start_row):
        if len(row) >= ITEM_MASTER_LIST_COL and row[ITEM_MASTER_LIST_COL - 1].strip():
            product_name = clean_product_name(row[ITEM_MASTER_LIST_COL - 1].strip())
            existing_products.append({"name": product_name, "row": row_idx})

    existing_suppliers = {}
    for col_idx in range(
        ITEM_MASTER_LIST_COL,
        len(sheet_values[0]) if sheet_values else 0,
        COLUMNS_PER_SUPPLIER,
    ):
        if COMPANY_NAME_ROW - 1 < len(sheet_values) and col_idx < len(
            sheet_values[COMPANY_NAME_ROW - 1]
        ):
            supplier_name = sheet_values[COMPANY_NAME_ROW - 1][col_idx].strip()
            if supplier_name:
                existing_suppliers[supplier_name] = col_idx

    next_avail_col = find_next_available_column(ws)

    for data in all_json_data:
        products = data.get("products", [])
        if not products:
            continue

        for product in products:
            if product.get("name"):
                product["name"] = clean_product_name(product["name"])

        company_name = data.get("company", "Unknown Company")

        col_idx = existing_suppliers.get(company_name, next_avail_col)
        if col_idx == next_avail_col:
            next_avail_col += COLUMNS_PER_SUPPLIER

        batch_requests = [
            {
                "range": f"{get_column_letter(col_idx)}{COMPANY_NAME_ROW}",
                "values": [[company_name]],
            },
            {
                "range": f"{get_column_letter(col_idx)}{CONTACT_INFO_ROW}",
                "values": [[f"{data.get('contact','')}".strip()]],
            },
            {
                "range": f"{get_column_letter(col_idx)}{HEADER_ROW}:{get_column_letter(col_idx+COLUMNS_PER_SUPPLIER-1)}{HEADER_ROW}",
                "values": [["ปริมาณ", "หน่วย", "ราคาต่อหน่วย", "รวมเป็นเงิน"]],
            },
        ]

        reference_data = [{"name": item["name"]} for item in existing_products]
        match_results = match_products_with_gemini(products, reference_data)
        matched_items = match_results["matchedItems"]
        unique_items = match_results["uniqueItems"]

        populated_rows = set()

        for item in matched_items:
            item_name = item["name"]
            for existing in existing_products:
                if (
                    existing["name"] == item_name
                    and existing["row"] not in populated_rows
                ):
                    batch_requests.append(
                        {
                            "range": f"{get_column_letter(col_idx)}{existing['row']}:{get_column_letter(col_idx+COLUMNS_PER_SUPPLIER-1)}{existing['row']}",
                            "values": [
                                [
                                    item.get("quantity", 1),
                                    item.get("unit", "ชิ้น"),
                                    item.get("pricePerUnit", 0),
                                    item.get("totalPrice", 0),
                                ]
                            ],
                        }
                    )
                    populated_rows.add(existing["row"])
                    break

        new_products = []
        for item in unique_items:
            if isinstance(item, dict) and "name" in item and "quantity" in item:
                item["name"] = clean_product_name(item["name"])

                if not any(
                    existing["name"] == item["name"] for existing in existing_products
                ):
                    new_products.append(item)

        next_row = start_row + len(existing_products)

        if new_products:
            new_rows = []
            for _ in range(len(new_products)):
                new_rows.append([""] * ws.col_count)

            if new_rows:
                ws.insert_rows(new_rows, next_row)

            for i, product in enumerate(new_products):
                row = next_row + i
                batch_requests.append(
                    {
                        "range": f"{get_column_letter(ITEM_MASTER_LIST_COL)}{row}",
                        "values": [[product.get("name", "Unknown Product")]],
                    }
                )

                batch_requests.append(
                    {
                        "range": f"{get_column_letter(col_idx)}{row}:{get_column_letter(col_idx+COLUMNS_PER_SUPPLIER-1)}{row}",
                        "values": [
                            [
                                product.get("quantity", 1),
                                product.get("unit", "ชิ้น"),
                                product.get("pricePerUnit", 0),
                                product.get("totalPrice", 0),
                            ]
                        ],
                    }
                )

                existing_products.append(
                    {"name": product.get("name", "Unknown Product"), "row": row}
                )

        summary_row = start_row + len(existing_products) + 2
        summary_items = [
            ("รวมเป็นเงิน", data.get("totalPrice", 0)),
            ("ภาษีมูลค่าเพิ่ม 7%", data.get("totalVat", 0)),
            ("ยอดรวมทั้งสิ้น", data.get("totalPriceIncludeVat", 0)),
            ("กำหนดยืนราคา (วัน)", data.get("priceGuaranteeDay", "")),
            ("ระยะเวลาส่งมอบสินค้าหลังจากได้รับ PO", data.get("deliveryTime", "")),
            ("การชำระเงิน", data.get("paymentTerms", "")),
            ("อื่น ๆ", data.get("otherNotes", "")),
        ]

        for i, (label, value) in enumerate(summary_items):
            row = summary_row + i
            batch_requests.append(
                {
                    "range": f"{get_column_letter(ITEM_MASTER_LIST_COL)}{row}",
                    "values": [[label]],
                }
            )
            batch_requests.append(
                {
                    "range": f"{get_column_letter(col_idx+COLUMNS_PER_SUPPLIER-1)}{row}",
                    "values": [[value]],
                }
            )

        if batch_requests:
            ws.batch_update(batch_requests, value_input_option="USER_ENTERED")

        if col_idx not in existing_suppliers.values():
            existing_suppliers[company_name] = col_idx

    return len(all_json_data)


def get_file_type(file_path):
    mime_type, _ = mimetypes.guess_type(file_path)

    if mime_type:
        if mime_type.startswith("image/"):
            return "image"
        elif mime_type == "application/pdf":
            return "pdf"
        elif mime_type in [
            "application/msword",
            "application/vnd.openxmlformats-officedocument.wordprocessingml.document",
        ]:
            return "word"

    ext = os.path.splitext(file_path)[1].lower()
    if ext in [".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".gif"]:
        return "image"
    elif ext == ".pdf":
        return "pdf"
    elif ext in [".doc", ".docx"]:
        return "word"

    return "unknown"


def process_file(file_path):
    file_type = get_file_type(file_path)
    tmp = tempfile.NamedTemporaryFile(
        delete=False, suffix=os.path.splitext(file_path)[1]
    )
    with open(file_path, "rb") as src:
        tmp.write(src.read())
    tmp.close()
    f = genai.upload_file(path=tmp.name, display_name=os.path.basename(file_path))
    if file_type == "image":
        model = genai.GenerativeModel(
            model_name="gemini-2.5-pro",
            generation_config={"temperature": 0.1, "top_p": 0.95},
        )
        resp = model.generate_content([image_prompt, f])
    else:
        model = genai.GenerativeModel(
            model_name="gemini-2.5-pro",
            generation_config={"temperature": 0.1, "top_p": 0.95},
        )
        resp = model.generate_content([prompt, f])
    d = extract_json_from_text(resp.text)
    if not d or not d.get("products"):
        model_pro = genai.GenerativeModel(
            model_name="gemini-2.5-flash",
            generation_config={"temperature": 0.1, "top_p": 0.95},
        )
        resp_pro = model_pro.generate_content(
            [prompt if file_type != "image" else image_prompt, f]
        )
        d = extract_json_from_text(resp_pro.text)
    d = validate_json_data(d) if d else None
    d = enhance_with_gemini(d) if d else None
    os.unlink(tmp.name)
    return d


def process_files(file_paths, sheet_id=DEFAULT_SHEET_ID):
    data_list = []
    for p in file_paths:
        d = process_file(p)
        if d:
            data_list.append(d)
    if data_list:
        ws = authenticate_and_open_sheet(sheet_id)
        update_google_sheet_with_multiple_files(ws, data_list)
    return data_list


def process_pdfs(pdf_paths, sheet_id=DEFAULT_SHEET_ID):
    return process_files(pdf_paths, sheet_id)


app
import React, { useState, useRef, useCallback } from 'react';
import axios from 'axios';
import './App.css';

function App() {
  const [files, setFiles] = useState([]);
  const [sheetUrl, setSheetUrl] = useState('');
  const [isLoading, setIsLoading] = useState(false);
  const [message, setMessage] = useState('');
  const [processedData, setProcessedData] = useState(null);
  const [currentStep, setCurrentStep] = useState(1);
  const [isDragging, setIsDragging] = useState(false);
  const fileInputRef = useRef(null);
  const [sheetId, setSheetId] = useState('');
  const [processingStage, setProcessingStage] = useState('waiting'); // Added state for tracking stages

  const extractSheetId = (url) => {
    if (!url) return '';
    
    if (!url.includes('/')) return url;
    
    const regexPatterns = [
      /\/spreadsheets\/d\/([a-zA-Z0-9-_]+)/,
      /\/spreadsheets\/u\/\d+\/d\/([a-zA-Z0-9-_]+)/,
    ];
    
    for (const regex of regexPatterns) {
      const match = url.match(regex);
      if (match && match[1]) return match[1];
    }
    
    return '';
  };

  const handleDragEnter = (e) => {
    e.preventDefault();
    e.stopPropagation();
    setIsDragging(true);
  };

  const handleDragLeave = (e) => {
    e.preventDefault();
    e.stopPropagation();
    setIsDragging(false);
  };

  const handleDragOver = (e) => {
    e.preventDefault();
    e.stopPropagation();
  };

  const isValidFileType = (file) => {
    const validTypes = [
      'application/pdf', // PDF files
      'image/jpeg', 'image/jpg', 'image/png', 'image/gif', 'image/bmp', 'image/tiff', // Common image types
    ];
    return validTypes.includes(file.type);
  };

  const handleDrop = useCallback((e) => {
    e.preventDefault();
    e.stopPropagation();
    setIsDragging(false);
    
    if (e.dataTransfer.files && e.dataTransfer.files.length > 0) {
      const droppedFiles = Array.from(e.dataTransfer.files).filter(isValidFileType);
      
      if (droppedFiles.length > 0) {
        setFiles(prevFiles => [...prevFiles, ...droppedFiles]);
      } else {
        setMessage('กรุณาอัปโหลดไฟล์ PDF หรือรูปภาพเท่านั้น');
      }
    }
  }, []);

  const handleFileChange = (e) => {
    if (e.target.files && e.target.files.length > 0) {
      const selectedFiles = Array.from(e.target.files).filter(isValidFileType);
      if (selectedFiles.length > 0) {
        setFiles(prevFiles => [...prevFiles, ...selectedFiles]);
      } else {
        setMessage('กรุณาอัปโหลดไฟล์ PDF หรือรูปภาพเท่านั้น');
      }
    }
  };

  const handleBrowseClick = () => {
    fileInputRef.current.click();
  };

  const removeFile = (indexToRemove) => {
    setFiles(files.filter((_, index) => index !== indexToRemove));
  };

  const handleSheetUrlChange = (e) => {
    const url = e.target.value;
    setSheetUrl(url);
    setSheetId(extractSheetId(url));
  };

  const handleSubmit = async (e) => {
    e.preventDefault();
    if (files.length === 0 || !sheetUrl) {
      setMessage('กรุณาเลือกไฟล์และกรอก Google Sheet URL');
      return;
    }

    setIsLoading(true);
    setMessage('');
    setProcessedData(null);
    setCurrentStep(2);
    setProcessingStage('extraction'); // Start with extraction stage
    
    const formData = new FormData();
    files.forEach(file => {
      formData.append('files', file);
    });
    formData.append('sheet_url', sheetUrl);

    try {
      // Simulate the different processing stages (in real app, this would come from backend updates)
      setTimeout(() => setProcessingStage('matching'), 2000);
      setTimeout(() => setProcessingStage('updating'), 4000);
      
      const response = await axios.post('http://localhost:8000/process-files/', formData, {
        headers: {
          'Content-Type': 'multipart/form-data'
        }
      });
      setMessage(response.data.message);
      setProcessedData(response.data.processed_data);
      setCurrentStep(3);
    } catch (error) {
      const errorMessage = error.response ? error.response.data.detail : error.message;
      setMessage(`เกิดข้อผิดพลาด: ${errorMessage}`);
      setCurrentStep(3);
    } finally {
      setIsLoading(false);
      setProcessingStage('waiting'); // Reset processing stage when done
    }
  };

  const navigateToStep = (stepNumber) => {
    setCurrentStep(stepNumber);
  };

  const formatDate = () => {
    const now = new Date();
    return now.toLocaleDateString('th-TH', {
      year: 'numeric',
      month: 'long',
      day: 'numeric',
      hour: '2-digit',
      minute: '2-digit'
    });
  };

  const getFileIcon = (file) => {
    if (file.type === 'application/pdf') {
      return (
        <svg className="file-icon" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">
          <path d="M14 2H6a2 2 0 0 0-2 2v16a2 2 0 0 0 2 2h12a2 2 0 0 0 2-2V8z"></path>
          <polyline points="14 2 14 8 20 8"></polyline>
          <line x1="16" y1="13" x2="8" y2="13"></line>
          <line x1="16" y1="17" x2="8" y2="17"></line>
          <polyline points="10 9 9 9 8 9"></polyline>
        </svg>
      );
    } else {
      return (
        <svg className="file-icon" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">
          <rect x="3" y="3" width="18" height="18" rx="2" ry="2"></rect>
          <circle cx="8.5" cy="8.5" r="1.5"></circle>
          <polyline points="21 15 16 10 5 21"></polyline>
        </svg>
      );
    }
  };

  const renderStepContent = () => {
    switch(currentStep) {
      case 1:
        return (
          <section className="form-step">
            <h2 className="font-normal">อัปโหลดข้อมูล</h2>
            <div className="upload-section">
              <div className="upload-header">
              </div>

              <div className="form-group sheet-input">
                <label htmlFor="sheetUrl">Google Sheet URL or ID:</label>
                <input
                  type="text"
                  id="sheetUrl"
                  value={sheetUrl}
                  onChange={handleSheetUrlChange}
                  placeholder="ใส่ URL หรือ ID ของ Google Sheet"
                  required
                />
              </div>

              <div 
                className={`file-drop-area ${isDragging ? 'dragging' : ''}`}
                onDragEnter={handleDragEnter}
                onDragLeave={handleDragLeave}
                onDragOver={handleDragOver}
                onDrop={handleDrop}
              >
                <div className="drop-icon">
                  <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" fill="none" stroke="currentColor" strokeWidth="2" strokeLinecap="round" strokeLinejoin="round">
                    <path d="M21 15v4a2 2 0 0 1-2 2H5a2 2 0 0 1-2-2v-4"></path>
                    <polyline points="17 8 12 3 7 8"></polyline>
                    <line x1="12" y1="3" x2="12" y2="15"></line>
                  </svg>
                </div>
                <p className="drop-text">Choose a file or drag & drop it here</p>
                <p className="file-formats">PDF or image files (JPG, PNG, etc.), up to 50MB</p>
                <input
                  ref={fileInputRef}
                  type="file"
                  accept=".pdf,.jpg,.jpeg,.png,.gif,.bmp,.tiff"
                  onChange={handleFileChange}
                  multiple
                  style={{ display: 'none' }}
                />
                <button 
                  type="button" 
                  className="browse-button"
                  onClick={handleBrowseClick}
                >
                  Browse File
                </button>
              </div>

              {files.length > 0 && (
                <div className="selected-files">
                  <h3>ไฟล์ที่เลือก:</h3>
                  <ul>
                    {files.map((file, index) => (
                      <li key={index}>
                        <div className="file-info">
                          {getFileIcon(file)}
                          <span className="file-name">{file.name}</span>
                          <span className="file-size">({(file.size / 1024).toFixed(1)} KB)</span>
                        </div>
                        <button 
                          className="remove-file" 
                          onClick={() => removeFile(index)}
                          title="Remove file"
                        >
                          <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" fill="none" stroke="currentColor" strokeWidth="2" strokeLinecap="round" strokeLinejoin="round">
                            <line x1="18" y1="6" x2="6" y2="18"></line>
                            <line x1="6" y1="6" x2="18" y2="18"></line>
                          </svg>
                        </button>
                      </li>
                    ))}
                  </ul>
                </div>
              )}
            </div>
            
            <div className="mt-3 action-buttons">
              <button 
                className="button submit-btn" 
                type="button" 
                onClick={handleSubmit}
                disabled={files.length === 0 || !sheetUrl}
              >
                เริ่มประมวลผล
              </button>
            </div>
          </section>
        );
      case 2:
        return (
          <section className="form-step loading-step">
            <div className="loading-container">
              <div className="spinner"></div>
              <h2>กำลังประมวลผลข้อมูล...</h2>
              <p>กรุณารอสักครู่ ระบบกำลังประมวลผลไฟล์ของคุณ</p>
              {files.length > 0 && (
                <div className="processing-files">
                  <p>กำลังประมวลผลไฟล์: {files.length} ไฟล์</p>
                  <div className="file-names">
                    {files.map((file, index) => (
                      <span key={index} className="processing-file-name">
                        {file.name}
                        {index < files.length - 1 ? ', ' : ''}
                      </span>
                    ))}
                  </div>
                </div>
              )}
              
              {/* Processing stages indicator */}
              <div className="processing-stages">
                <div className={`stage ${processingStage === 'extraction' ? 'active' : processingStage === 'matching' || processingStage === 'updating' ? 'completed' : ''}`}>
                  <div className="stage-icon">
                    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" fill="none" stroke="currentColor" strokeWidth="2" strokeLinecap="round" strokeLinejoin="round">
                      <path d="M14 2H6a2 2 0 0 0-2 2v16a2 2 0 0 0 2 2h12a2 2 0 0 0 2-2V8z"></path>
                      <polyline points="14 2 14 8 20 8"></polyline>
                      <line x1="16" y1="13" x2="8" y2="13"></line>
                      <line x1="16" y1="17" x2="8" y2="17"></line>
                    </svg>
                  </div>
                  <div className="stage-label">Data Extraction</div>
                </div>
                
                <div className="stage-connector"></div>
                
                <div className={`stage ${processingStage === 'matching' ? 'active' : processingStage === 'updating' ? 'completed' : ''}`}>
                  <div className="stage-icon">
                    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" fill="none" stroke="currentColor" strokeWidth="2" strokeLinecap="round" strokeLinejoin="round">
                      <path d="M22 11.08V12a10 10 0 1 1-5.93-9.14"></path>
                      <polyline points="22 4 12 14.01 9 11.01"></polyline>
                    </svg>
                  </div>
                  <div className="stage-label">Data Matching</div>
                </div>
                
                <div className="stage-connector"></div>
                
                <div className={`stage ${processingStage === 'updating' ? 'active' : ''}`}>
                  <div className="stage-icon">
                    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" fill="none" stroke="currentColor" strokeWidth="2" strokeLinecap="round" strokeLinejoin="round">
                      <path d="M12 2L2 7l10 5 10-5-10-5z"></path>
                      <path d="M2 17l10 5 10-5"></path>
                      <path d="M2 12l10 5 10-5"></path>
                    </svg>
                  </div>
                  <div className="stage-label">Update to Google Sheet</div>
                </div>
              </div>
            </div>
          </section>
        );
      case 3:
        return (
          <section className="form-step result-step">
            <h2 className="font-normal">ผลการประมวลผล</h2>
            <div className="process-info">
              </div>

              <div className="mt-3">
                {message && (
                  <div className={`message ${processedData ? 'success' : 'error'}`}>
                    <p>{message}</p>
                  </div>
                )}
              </div>

            {processedData && (
              <div className="results-container">
                <div className="results-tabs">
                  <button className="tab-button active">ข้อมูลที่ประมวลผล</button>
                  <button className="tab-button">Google Sheet</button>
                </div>
                
                <div className="tab-content">
                  <div className="data-preview">
                    <h3>ข้อมูลที่ประมวลผลได้:</h3>
                    <pre>{JSON.stringify(processedData, null, 2)}</pre>
                  </div>

                  {sheetId && (
                    <div className="sheet-embed">
                      <h3>Google Sheet ที่อัปเดตข้อมูล:</h3>
                      <div className="google-sheet-container">
                        <iframe 
                          title="Google Sheet Data"
                          src={`https://docs.google.com/spreadsheets/d/${sheetId}/htmlembed?widget=true`}
                          className="google-sheet-iframe"
                        ></iframe>
                      </div>
                    </div>
                  )}
                </div>
              </div>
            )}
            
            <div className="mt-3 action-buttons">
              <button className="button" type="button" onClick={() => {
                setFiles([]);
                setSheetUrl('');
                setMessage('');
                setProcessedData(null);
                navigateToStep(1);
              }}>เริ่มใหม่</button>

              {sheetId && (
                <a 
                  href={`https://docs.google.com/spreadsheets/d/${sheetId}/edit`}
                  target="_blank"
                  rel="noopener noreferrer"
                  className="button view-sheet-btn"
                >
                  เปิด Google Sheet
                </a>
              )}
            </div>
          </section>
        );
      default:
        return null;
    }
  };

  return (
    <div className="container">
      <h1>ระบบประมวลผลใบเสนอราคา V1.3</h1>
      <p>อัปโหลดไฟล์ใบเสนอราคา (PDF หรือรูปภาพ) เพื่อประมวลผลและส่งข้อมูลเข้า Google Sheet</p>
      
      <div id="multi-step-form-container">
        <ul className="form-stepper form-stepper-horizontal text-center mx-auto pl-0">
          <li className={`text-center form-stepper-list ${currentStep === 1 ? 'form-stepper-active' : currentStep > 1 ? 'form-stepper-completed' : 'form-stepper-unfinished'}`} step="1">
            <a className="mx-2">
              <span className="form-stepper-circle">
                <span>1</span>
              </span>
              <div className="label">เลือกไฟล์</div>
            </a>
          </li>
          <li className={`text-center form-stepper-list ${currentStep === 2 ? 'form-stepper-active' : currentStep > 2 ? 'form-stepper-completed' : 'form-stepper-unfinished'}`} step="2">
            <a className="mx-2">
              <span className="form-stepper-circle">
                <span>2</span>
              </span>
              <div className="label">ประมวลผล</div>
            </a>
          </li>
          <li className={`text-center form-stepper-list ${currentStep === 3 ? 'form-stepper-active' : 'form-stepper-unfinished'}`} step="3">
            <a className="mx-2">
              <span className="form-stepper-circle">
                <span>3</span>
              </span>
              <div className="label">ผลการประมวลผล</div>
            </a>
          </li>
        </ul>
        <form id="userAccountSetupForm" name="userAccountSetupForm" encType="multipart/form-data">
          {renderStepContent()}
        </form>
      </div>
    </div>
  );
}

export default App;

In [ ]:
prompt = """# System Message for Product List Extraction (PDF/Text Table Processing)
## CRITICAL: ANTI-HALLUCINATION WARNING
You MUST ONLY extract information that is EXPLICITLY visible in the document. 
- DO NOT add data from anywhere other than the one in the uploaded document. Adding data that is not from the document is a serious mistake.
- DO NOT create or invent any products, prices, or specifications
- DO NOT add data from the attached Example in the prompt.
- DO NOT add products that are not clearly listed as distinct line items
- DO NOT attempt to break down a single product into multiple products
- DO NOT interpret descriptive text as separate products
- If uncertain about any information, LEAVE IT OUT rather than guessing

## CRITICAL: CONTACT INFORMATION EXTRACTION
Pay special attention to contact information in the document header or footer:
- Extract any email addresses (example@domain.com)
- Extract any phone numbers (formats like 02-3384825, 081-1234567, 095-525-2623)
- Put email and phone details in the "contact" field
- Format: "Email: email@example.com, Phone: 081-234-5678"
- Use only the phone number and email address on the letterhead. Do not add "บริษัท ลูก้า แอสเซท จำกัด, 081-781-7283" contact information

## CRITICAL: COMPLETE EXTRACTION REQUIREMENT
You MUST extract ALL products visible in the document:
- Extract EVERY product line item visible in the document
- Preserve hierarchical structure (groups/categories) of products if present
- Ensure NO products are missed or skipped
- Each row with a distinct price is one product

## Input Format
Provide PDF files (or images/tables with text extraction) containing product information (receipts, invoices, product lists, etc.).

## Task
Extract ALL product information EXPLICITLY visible in the document:
- Extract products with quantities, units, and prices
- Preserve parent-child relationships (main categories and sub-items)
- Maintain hierarchical product groupings (numbered sections, categories)
- Also extract additional quotation details like price validity, delivery time, payment terms, etc.

## Hierarchical Structure Handling
Many quotations organize products hierarchically. When you see this:
- Include category names in product descriptions (e.g., "งานบันไดกระจก งานพื้นตก - กระจกเทมเปอร์ใส หนา 10 มม. ขนาด 4.672×0.97 ม.")
- If product descriptions begin with numbers (1, 2, 3...), REMOVE those numbers
- Include all parent category information in each product's name without the leading numbers

## Output Format (JSON only)
You must return ONLY this JSON structure:
{
  "company": "company name or first name + last name (NEVER null)",
  "vat": true,
  "name": "customer name or null",
  "contact": "phone number or email or null",
  "priceGuaranteeDay": 30
  "deliveryTime": "",
  "paymentTerms": "",
  "otherNotes": "",
  "products": [
    {
      "name": "full product description including ALL parent category info, specifications AND dimensions WITHOUT leading numbers",
      "quantity": 1,
      "unit": "match the unit shown in the document (e.g., แผ่น, ตร.ม., ชิ้น, ตัว, เมตร, ชุด)",
      "pricePerUnit": 0,
      "totalPrice": 0
    }
  ],
  "totalPrice": 0,
  "totalVat": 0,
  "totalPriceIncludeVat": 0
}

## Example 1 (Format with Item/ART.No./Description/Qty/Unit/Price columns):
| Item | ART.No. | Description | Qty | Unit | Standard Price | Discount Price | Amount |
|------|---------|-------------|-----|------|----------------|---------------|--------|
| 1    | CPW-xxxx| SPC ลายไม้ 4.5 มิล (ก้างปลา) | 1.00 | ตร.ม. |  | 520.00 | 520.000 |
| 2    |         | ค่าแรงติดตั้ง | 1.00 | ตร.ม. |  | 150.00 | 150.000 |

## Example 2 (Format with ลำดับ/รหัสสินค้า/รายละเอียดสินค้า columns):
| ลำดับ | รหัสสินค้า | รายละเอียดสินค้า | หน่วย | จำนวน | ราคา/หน่วย(บาท) | จำนวนเงิน(บาท) |
|------|---------|----------------|------|------|--------------|------------|
| 1    |         | พื้นไม้ไวนิลลายไม้ปลา 4.5 มม. LKT 4.5 mm x 0.3 mm สีฟ้าเซอร์คูลี (1 กล่อง บรรจุ 18 แผ่น หรือ 1.3 ตร.ม) | ตร.ม. | 1.30 | 680.00 | 884.00 |

## Field Extraction Guidelines

### name (Product Description)
* CRITICAL: Include ALL hierarchical information in each product name:
  - Category names/headings (e.g., "งานบันไดกระจก งานพื้นตก")
  - Sub-category information (e.g., "เหล็กตัวซีชุบสังกะสี")
  - Glass type, thickness (e.g., "กระจกเทมเปอร์ใส หนา 10 มม.")
  - Exact dimensions (e.g., "ขนาด 4.672×0.97 ม.")
* REMOVE any leading numbers (1., 2., 3.) from the product descriptions
* Format hierarchical products as: "[Category Name] - [Material] - [Type] - [Dimensions]"
* Include: ALL distinguishing characteristics that make each product unique
* Example: "งานบันไดกระจก งานพื้นตก - เหล็กตัวซีชุบสังกะสี ไม่รวมปูน - กระจกเทมเปอร์ใส หนา 10 มม. ขนาด 4.672×0.97 ม."

### unit and quantity (DIRECT EXTRACTION RULE)
* Extract unit and quantity DIRECTLY from each line item as shown
* Use the exact unit shown in the document (ชุด, แผ่น, ตร.ม., ชิ้น, ตัว, เมตร, etc.)
* Extract the exact quantity shown for each product (never assume or calculate)
* NEVER create quantities or units that aren't explicitly shown in the document
* Pay special attention to decimal quantities - extract the full decimal precision

### pricePerUnit and totalPrice
* Extract ONLY prices clearly visible in the document
* Use numeric values only (no currency symbols)
* Extract cleanly from pricing fields as shown in each line item
* NEVER calculate or estimate prices that aren't explicitly shown
* NEVER combine different products' prices
* Pay special attention to decimal prices - extract the EXACT decimal values shown

### Additional Quotation Details
* Extract these additional fields if present:
  - "กำหนดยืนราคา (วัน)", "กำหนดยืนราคา", "การยืนราคา" - Price validity period in days (priceGuaranteeDay)
  - "ระยะเวลาส่งมอบสินค้าหลังจากได้รับ PO" - Delivery time after PO (deliveryTime)
  - "การชำระเงิน" - Payment terms (paymentTerms)
  - "อื่น ๆ" - Other notes (otherNotes)
* Extract as text exactly as written, preserving numbers and Thai language

### CRITICAL: Pricing summaries and summary values
* Extract the exact values for these three summary items:
  - "รวมเป็นเงิน" - the initial subtotal (totalPrice)
  - "ภาษีมูลค่าเพิ่ม 7%" - the VAT amount (totalVat)
  - "ยอดรวมทั้งสิ้น" - the final total (totalPriceIncludeVat)
* Alternative labels to match:
  - For totalPrice: "รวม", "รวมเป็นเงิน", "ราคารวม", "Total", "TOTAL AMOUNT", "รวมราคา"
  - For totalVat: "ภาษีมูลค่าเพิ่ม 7%", "VAT 7%"
  - For totalPriceIncludeVat: "ยอดรวมทั้งสิ้น", "รวมทั้งหมด", "รวมเงินทั้งสิน", "ราคารวมสุทธิ", "รวมราคางานทั้งหมดตามสัญญา"
* Extract the exact values as shown (remove commas, currency symbols)
* CRITICAL: Preserve full decimal precision in all monetary values

## FINAL VERIFICATION
Review the extracted products one last time and verify:
1. Count the number of products you've extracted
2. Verify this matches EXACTLY with the number of product rows visible in the document
3. Check that ALL products have proper hierarchical information included WITHOUT leading numbers
4. Ensure NO products are missing - every line item with a price must be extracted
5. Confirm all dimensions and specifications are preserved correctly
6. Verify all decimal values (quantities and prices) maintain their full precision
"""

image_prompt = """# System Message for Product List Extraction from Images
## CRITICAL: ANTI-HALLUCINATION WARNING
You MUST ONLY extract information that is EXPLICITLY visible in the image. 
- DO NOT create or invent any products, prices, or specifications
- DO NOT add products that are not clearly listed as distinct line items
- DO NOT interpret descriptive text as separate products
- If text is unclear or unreadable, mark it as uncertain rather than guessing

## CRITICAL: COMPLETE EXTRACTION REQUIREMENT
You MUST extract ALL products visible in the image:
- Extract EVERY product line item visible in the image
- Preserve hierarchical structure (groups/categories) of products if present
- Ensure NO products are missed or skipped
- Each row with a distinct price is one product

## Input Format
I'm providing an image of a document containing product information.

## Task
Extract ALL product information EXPLICITLY visible in the image:
- Extract products with quantities, units, and prices
- Preserve parent-child relationships (main categories and sub-items)
- Maintain hierarchical product groupings (numbered sections, categories)
- Also extract additional quotation details like price validity, delivery time, payment terms, etc.

## Hierarchical Structure Handling
Many quotations organize products hierarchically. When you see this:
- Include category names in product descriptions (e.g., "งานบันไดกระจก งานพื้นตก - กระจกเทมเปอร์ใส หนา 10 มม. ขนาด 4.672×0.97 ม.")
- If product descriptions begin with numbers (1, 2, 3...), REMOVE those numbers
- Include all parent category information in each product's name without the leading numbers

## Output Format (JSON only)
You must return ONLY this JSON structure:
{
  "company": "company name or first name + last name (NEVER null)",
  "vat": true,
  "contact": "phone number or email or null",
  "priceGuaranteeDay": 30
  "deliveryTime": "",
  "paymentTerms": "",
  "otherNotes": "",
  "products": [
    {
      "name": "full product description including ALL parent category info, specifications AND dimensions WITHOUT leading numbers",
      "quantity": 1,
      "unit": "match the unit shown in the document (e.g., แผ่น, ตร.ม., ชิ้น, ตัว, เมตร, ชุด)",
      "pricePerUnit": 0,
      "totalPrice": 0
    }
  ],
  "totalPrice": 0,
  "totalVat": 0,
  "totalPriceIncludeVat": 0
}


## Example 1 (Format with Item/ART.No./Description/Qty/Unit/Price columns):
| Item | ART.No. | Description | Qty | Unit | Standard Price | Discount Price | Amount |
|------|---------|-------------|-----|------|----------------|---------------|--------|
| 1    | CPW-xxxx| SPC ลายไม้ 4.5 มิล (ก้างปลา) | 1.00 | ตร.ม. |  | 520.00 | 520.000 |
| 2    |         | ค่าแรงติดตั้ง | 1.00 | ตร.ม. |  | 150.00 | 150.000 |

## Example 2 (Format with ลำดับ/รหัสสินค้า/รายละเอียดสินค้า columns):
| ลำดับ | รหัสสินค้า | รายละเอียดสินค้า | หน่วย | จำนวน | ราคา/หน่วย(บาท) | จำนวนเงิน(บาท) |
|------|---------|----------------|------|------|--------------|------------|
| 1    |         | พื้นไม้ไวนิลลายไม้ปลา 4.5 มม. LKT 4.5 mm x 0.3 mm สีฟ้าเซอร์คูลี (1 กล่อง บรรจุ 18 แผ่น หรือ 1.3 ตร.ม) | ตร.ม. | 1.30 | 680.00 | 884.00 |


## Field Extraction Guidelines

### name (Product Description)
* CRITICAL: Include ALL hierarchical information in each product name:
  - Category names/headings (e.g., "งานบันไดกระจก งานพื้นตก")
  - Sub-category information (e.g., "เหล็กตัวซีชุบสังกะสี")
  - Glass type, thickness (e.g., "กระจกเทมเปอร์ใส หนา 10 มม.")
  - Exact dimensions (e.g., "ขนาด 4.672×0.97 ม.")
* REMOVE any leading numbers (1., 2., 3.) from the product descriptions
* Format hierarchical products as: "[Category Name] - [Material] - [Type] - [Dimensions]"
* Include: ALL distinguishing characteristics that make each product unique
* Example: "งานบันไดกระจก งานพื้นตก - เหล็กตัวซีชุบสังกะสี ไม่รวมปูน - กระจกเทมเปอร์ใส หนา 10 มม. ขนาด 4.672×0.97 ม."

### unit and quantity (DIRECT EXTRACTION RULE)
* Extract unit and quantity DIRECTLY from each line item as shown
* Use the exact unit shown in the document (ชุด, แผ่น, ตร.ม., ชิ้น, ตัว, เมตร, จำนวนต่อชุด etc.)
* Extract the exact quantity shown for each product (never assume or calculate)
* NEVER create quantities or units that aren't explicitly shown in the document
* Pay special attention to decimal quantities - extract the full decimal precision

### pricePerUnit and totalPrice
* Extract ONLY prices clearly visible in the document
* Use numeric values only (no currency symbols)
* Extract cleanly from pricing fields as shown in each line item
* NEVER calculate or estimate prices that aren't explicitly shown
* NEVER combine different products' prices
* Pay special attention to decimal prices - extract the EXACT decimal values shown

### Additional Quotation Details
* Extract these additional fields if present:
  - "กำหนดยืนราคา (วัน)", "กำหนดยืนราคา", "การยืนราคา" - Price validity period in days (priceGuaranteeDay)
  - "ระยะเวลาส่งมอบสินค้าหลังจากได้รับ PO" - Delivery time after PO (deliveryTime)
  - "การชำระเงิน" - Payment terms (paymentTerms)
  - "อื่น ๆ" - Other notes (otherNotes)
* Extract as text exactly as written, preserving numbers and Thai language

### CRITICAL: Pricing summaries and summary values
* Extract the exact values for these three summary items:
  - "รวมเป็นเงิน" - the initial subtotal (totalPrice)
  - "ภาษีมูลค่าเพิ่ม 7%" - the VAT amount (totalVat)
  - "ยอดรวมทั้งสิ้น" - the final total (totalPriceIncludeVat)
* Alternative labels to match:
  - For totalPrice: "รวม", "รวมเป็นเงิน", "ราคารวม", "Total", "TOTAL AMOUNT", "รวมราคา"
  - For totalVat: "ภาษีมูลค่าเพิ่ม 7%", "VAT 7%"
  - For totalPriceIncludeVat: "ยอดรวมทั้งสิ้น", "รวมทั้งหมด", "รวมเงินทั้งสิน", "ราคารวมสุทธิ", "รวมราคางานทั้งหมดตามสัญญา"
* Extract the exact values as shown (remove commas, currency symbols)
* CRITICAL: Preserve full decimal precision in all monetary values

## FINAL VERIFICATION
Review the extracted products one last time and verify:
1. Count the number of products you've extracted
2. Verify this matches EXACTLY with the number of product rows visible in the image
3. Check that ALL products have proper hierarchical information included WITHOUT leading numbers
4. Ensure NO products are missing - every line item with a price must be extracted
5. Confirm all dimensions and specifications are preserved correctly
6. Verify all decimal values (quantities and prices) maintain their full precision
"""

validation_prompt = """
You are a data validation expert specializing in Thai construction quotations.
I've extracted product data from a document, but there may be missing products or hierarchical relationships.

## CRITICAL: COMPLETE DATA CHECK
Your primary task is to ensure ALL products are correctly extracted with their hierarchical structure:
1. Check that all products visible in the document have been extracted
2. Ensure parent-child relationships and category groupings are preserved
3. Verify that all products have complete descriptions including their category name
4. Make sure no products are missing dimensions or specifications

## CRITICAL: PRESERVE PRODUCT HIERARCHY
Thai construction quotations often organize products hierarchically by categories:
- Category names with descriptive details
- Materials and specifications
- Dimensions

Each product must include its complete hierarchy:
"[Category Name] - [Material] - [Type] - [Dimensions]"

Examples (DO NOT add data from the attached Example in the prompt): 
- "งานบันไดกระจก งานพื้นตก - เหล็กตัวซีชุบสังกะสี ไม่รวมปูน - กระจกเทมเปอร์ใส หนา 10 มม. ขนาด 4.672×0.97 ม."
- "งานพื้นตก (ชั้นลอย) - เหล็กตัวซีชุบสังกะสี ไม่รวมปูน - เทมเปอร์ใส หนา 10 มม. ขนาด 3.565×0.97 ม."

## CRITICAL: DECIMAL NUMBER ACCURACY
Pay special attention to:
1. Quantities with decimals (extract full precision)
2. Dimensions with decimals (preserve exact measurements)
3. Prices with decimals (maintain exact values)

## CRITICAL: CLEAN PRODUCT DESCRIPTIONS
1. REMOVE any leading numbers (1., 2., 3., etc.) from product descriptions
2. Ensure NO product descriptions begin with numbering
3. Maintain all other hierarchical information and details

Review the data carefully and FIX these issues:
1. ADD any missing products that should be extracted from the source document
2. FIX product names to include complete hierarchical information WITHOUT leading numbers
3. ENSURE all dimensions and specifications are preserved with full decimal precision
4. VERIFY every product has the correct quantity, unit, price and total with full decimal precision

Original extraction:
{extracted_json}

Return ONLY a valid JSON object with no explanations.
"""

matching_prompt = """
You are a meticulous data architect specializing in product ontology for construction and home appliance materials. Your primary mission is to analyze product lists from different suppliers, establish a single "canonical" master product name for each item, and then map all supplier variations to that canonical name.
Your logic must be hierarchical and rule-based. Follow this algorithm precisely.

### **Core Objective: Create and Match to a Canonical Name**

The "Canonical Name" is the single source of truth for a product. You must construct it using this strict format:
**`[Group] - [Normalized Product Type] - [Primary Model/Identifier]`**

-   **`[Group]`**: The project phase or room size (e.g., "1BR+2BR(57-70sqm.)", "2BR (90sqm.)"). This is the **highest-priority** matching key.
-   **`[Normalized Product Type]`**: The generic category of the product (e.g., "Hood", "Induction Hob", "Sink"). You must deduce this from various descriptions.
-   **`[Primary Model/Identifier]`**: The most specific model number available (e.g., "EL 60", "MWE 255 FI").

### **Mandatory 4-Step Matching Algorithm**

For every product you process, you must follow these steps in order:

#### **Step 1: Group Matching (Non-Negotiable Filter)**
-   This is the most critical step. Products can **ONLY** be considered a match if they belong to the **exact same `[Group]`**.
-   Example: A "Hood" from "1BR+2BR(57-70sqm.)" can **NEVER** match a "Hood" from "2BR (90sqm.)". They are distinct line items.
-   Recognize semantic equivalents for groups, e.g., "1 BEDROOM" is the same as "1BR+2BR(57-70sqm.)".

#### **Step 2: Product Type Normalization & Keyword Mapping**
-   After filtering by group, identify the core product type. You must normalize different supplier descriptions into one standard type.
-   Use this keyword map as your guide:
    -   **"Hood"**: `Slimline Hood`, `BI telescopic hood`, `HOOD PIAVE 60 XS`
    -   **"Induction Hob"**: `Induction Hob`, `Hob Electric`, `HOB INDUCTION`
    -   **"Microwave"**: `Built-in Microwave`, `Microwave Oven`, `MICROWAVE FMWO 25 NH I`
    -   **"Sink"**: `Undermount Sink`, `Sink Stainless Steel`, `SINK BXX 210-45`
    -   **"Tap"**: `Sink Single Tap`, `Tap`, `TAP LANNAR`

#### **Step 3: Specification & Model Analysis**
-   Once Group and Normalized Type match, use the model number and other specifications (`Model`, `Description` columns) to create the full canonical name and to confirm the match.
-   The model number itself does not have to be identical between suppliers if the Group and Normalized Type are a clear match. The model number's purpose is to create the *unique canonical name* for that row.

#### **Step 4: Construct Final Output**
-   Based on the matches found, generate the final JSON.

### **Critical Rules & Constraints**

1.  **Group is King:** If the group doesn't match, nothing else matters.
2.  **Type over Model:** A strong match on `Group` + `Normalized Product Type` is more important than a weak match on `Model` number.
3.  **One-to-One Mapping:** A reference item (a canonical name you create) can only be matched once per supplier list.
4.  **No Imagination:** Only use information explicitly present in the data. If you cannot confidently normalize a product type, classify it as unique.

### **Walkthrough Example: Matching "Hoods"**

**Goal:** Match the first item from all three suppliers.

1.  **Teka:**
    -   **Input:** Group=`1BR+2BR(57-70sqm.)`, Model=`EL 60`, Desc=`Slimline Hood`
    -   **Analysis:** Group is "1BR...". Type normalizes from "Slimline Hood" to **"Hood"**. Model is `EL 60`.
    -   **Canonical Name Created:** `1BR+2BR(57-70sqm.) - Hood - EL 60`

2.  **Hisense (Gorenje):**
    -   **Input:** Group=`1BR+2BR (57-70Sqm.)`, Product=`TH62E3X`, Desc=`BI telescopic hood...`
    -   **Analysis:** Group is "1BR...". It matches Teka's group. Type normalizes from "BI telescopic hood" to **"Hood"**. It matches the normalized type.
    -   **Conclusion:** This is a match for the same row.

3.  **Franke:**
    -   **Input:** Group=`1 BEDROOM`, Product Category=`Hood`, Mode=`PIAVE 60 XS`
    -   **Analysis:** Group "1 BEDROOM" is semantically identical to "1BR...". It matches. Type is explicitly `Hood`. It matches.
    -   **Conclusion:** This is also a match for the same row.

All three products are mapped to the canonical name `1BR+2BR(57-70sqm.) - Hood - EL 60`, and their respective data will be aligned on this single row in the final output.

### **Input & Output Format**

-   **Input:** `target_products` (from a new quotation) and `reference_products` (the existing master list of canonical names).
-   **Output:** You **MUST** return a JSON object with this exact structure:

{{
  "matchedItems": [
    {{
      "name": "The canonical reference name this product matched to.",
      "quantity": "target quantity",
      "unit": "target unit",
      "pricePerUnit": "target price per unit",
      "totalPrice": "target total price"
    }}
  ],
  "uniqueItems": [
    {{
      "name": "The full, descriptive name of the target product that could not be matched.",
      "quantity": "target quantity",
      "unit": "target unit",
      "pricePerUnit": "target price per unit",
      "totalPrice": "target total price"
    }}
  ]
}}

## Target Products:
{target_products}

## Reference Products:
{reference_products}
"""